## Noun Phrase Chunking of IMDB Movie Reviews
### DS 7337 - Natural Laguage Processing - Homework 6
#### George C. Sturrock
##### March 16, 2019
##### Homework 6 Objectives
1.  Evaluate text similarity of Amazon book search results by doing the following:

a.  Do a book search on Amazon. Manually copy the full book title (including subtitle) of each of the top 24 books listed in the first two pages of search results. 

b.  In Python, run one of the text-similarity measures covered in this course, e.g., cosine similarity. Compare each of the book titles, pairwise, to every other one. 

c.  Which two titles are the most similar to each other? Which are the most dissimilar? Where do they rank, among the first 24 results?


2.  Now evaluate using a major search engine.

a.  Enter one of the book titles from question 1a into Google, Bing, or Yahoo!. Copy the capsule of the first organic result and the 20th organic result. Take web results only (i.e., not video results), and skip sponsored results. 

b.  Run the same text similarity calculation that you used for question 1b on each of these capsules in comparison to the original query (book title). 

c.  Which one has the highest similarity measure? 

Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings. 


#### Import Libraries

In [3]:
import platform; print("Platform:", platform.platform())
import os
import sys; print("Python:", sys.version)
import requests; print("Requests:", requests.__version__)
from urllib import request; ("urllib:", request.__version__)
import warnings
warnings.filterwarnings('ignore')
#from time import sleep

#Web Scraping
import bs4; print("BeatifulSoup:", bs4.__version__)
from bs4 import BeautifulSoup
import json; print("json:", json.__version__)

#Python Basics
import numpy as np; print("Numpy Version:", np.__version__)
import pandas as pd; print("Pandas Version:", pd.__version__)
import seaborn as sns;

#NLP
import nltk; print("NLTK Version:", nltk.__version__)
#nltk.download('conll2000')
from nltk.corpus import conll2000

import spacy; print("Spacy Version:", spacy.__version__)
from spacy import displacy
#nlp = spacy.load('en')
#import en_core_web_sm; print("en_core_web_sm Version:", en_core_web_sm.__version__)
#nlp = en_core_web_sm.load()
import en_core_web_md; print("en_core_web_md Version:", en_core_web_md.__version__)
nlp = en_core_web_md.load()

import pattern; print("Pattern Version:", pattern.__version__)
from pattern.en import tag
from pattern.en import parsetree

Platform: Windows-10-10.0.14393-SP0
Python: 3.6.8 |Anaconda custom (64-bit)| (default, Feb 11 2019, 15:03:47) [MSC v.1915 64 bit (AMD64)]
Requests: 2.21.0
BeatifulSoup: 4.7.1
json: 2.0.9
Numpy Version: 1.15.4
Pandas Version: 0.24.1
NLTK Version: 3.4
Spacy Version: 2.0.16
en_core_web_md Version: 2.0.0
Pattern Version: 3.6


### Question 6.1
Despite Stephen Ambrose's propensity for plagiarism, "Band of Brothers" is one of my all-time favorite books.  An Amazon book search for this title returns related results for this title.  The results include several formats of the original book included paper back, Kindle, DVD, etc.  These redundant results were not included in the list below which includes the top twenty-five related results (not including the aforementioned duplicates).  Additionally, only book results were included in the list.  T-shirts appeared in the search results even though the Amazon book search was filtered for "Books".  

In [7]:
boblist = ["Band of Brothers: E Company, 506th Regiment, 101st Airborne from Normandy to Hitler's Eagle's Nest", 
          "Beyond Band of Brothers: The War Memoirs of Major Dick Winters", 
          "Brothers in Battle, Best of Friends: Two WWII Paratroopers from the Original Band of Brothers Tell Their Story",
          "Shifty's War: The Authorized Biography of Sergeant Darrell Powers, the Legendary Sharpshooter from the Band", 
          "Conversations with Major Dick Winters: Life Lessons from the Commander of the Band of Brothers", 
          "Easy Company Soldier: The Legendary Battles of a Sergeant from WW II's 'Band of Brothers'",
          "How Easy Company Became a Band of Brothers",
          "Biggest Brother: The Life Of Major Dick Winters, The Man Who Led The Band of Brothers", 
          "We Who Are Alive and Remain: Untold Stories from the Band of Brothers", 
          "The General's Driver: A Vietnam Soldier Remembers", 
          "Killing the SS: The Hunt for the Worst War Criminals in History (Bill O'Reilly's Killing Series)", 
          "The Allman Brothers Band Classic Memorabilia, 1969-76 (Music and the American South)", 
          "Building Your Band of Brothers", 
          "Killer Genius: The Bizarre Case of the Homicidal Scholar (Dead True Crime Book 5)", 
          "Murderer's Gulch: Carnage in the Catskills (Dead True Crime Book 4)", 
          "Band of Brothers: Battle Manual", 
          "Call of Duty: My Life Before, During and After the Band of Brothers", 
          "Band of Brothers: The Complete Campaigns", 
          "My Kid Brother's Band... a.k.a. The Beatles", 
          "One More Moment (Shaughnessy Brothers: Band on the Run Book 3)", 
          "One Way Out: The Inside History of the Allman Brothers Band", 
          "Tangled Up in You (The Shaughnessy Brothers Book 7)", 
          "The Allman Brothers Band: The Definitive Collection for Guitar, Vol. 3", 
          "The Allman Brothers Band - The Definitive Collection for Guitar - Volume 1 (Guitar Recorded Versions S)", 
          "Bastogne Band of Brothers Guide", 
          "Daisy Jones & The Six: A Novel", 
          "Band of Brothers: E Company 506th Regiment 101st Airborne, D-day & Citizen Soldiers Box Set", 
          "D DAY Through German Eyes - The Hidden Story of June 6th 1944"]

#### Sarkar

In [12]:
# Sarker p. 270

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def build_feature_matrix(documents, feature_type = 'frequency',
                         ngram_range = (1, 1), min_df = 0.0, max_df = 1.0):

    feature_type = feature_type.lower().strip()  
    
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary = True, min_df = min_df,
                                     max_df = max_df, ngram_range = ngram_range)
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary = False, min_df = min_df,
                                     max_df = max_df, ngram_range = ngram_range)
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df = min_df, max_df = max_df, 
                                     ngram_range = ngram_range)
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")

    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
    return vectorizer, feature_matrix

In [13]:
# Sarker p. 287

def compute_cosine_similarity(doc_features, corpus_features, top_n = 3):

    # get document vectors
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()

    # compute similarities
    similarity = np.dot(doc_features, 
                        corpus_features.T)

    # get docs with highest similarity scores
    top_docs = similarity.argsort()[::-1][:top_n]
    
    top_docs_with_score = [(index, round(similarity[index], 3))
                            for index in top_docs]
    
    return top_docs_with_score

In [17]:
# Sarkar p. 206
from normalization import normalize_corpus


# Sarker p. 286-87

query_docs = boblist

#html_parser = HTMLParser()

norm_boblist = normalize_corpus(boblist, lemmatize = True)

tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_boblist,
                                                        feature_type = 'tfidf',
                                                        ngram_range = (1, 1), 
                                                        min_df = 0.0,
                                                        max_df = 1.0)
                                                        
# normalize and extract features from the query boblist

norm_query_docs =  normalize_boblist(query_docs, lemmatize = True)    
  

query_docs_tfidf = tfidf_vectorizer.transform(norm_query_docs)

# cosine similarity
    
print ('='*60)
print ('Book Titles Similarity Analysis using Cosine Similarity')
print ('='*60)
print ("\n")

# return top 2 similar, then drop the 1st if same as primary document

for index, doc in enumerate(query_docs):
    
    doc_tfidf = query_docs_tfidf[index]
    
    top_similar_docs = compute_cosine_similarity(doc_tfidf, tfidf_features, top_n = 27)
    
    if(top_similar_docs[0][0] == index) :
        tsd = top_similar_docs[1]
    else :
        tsd = top_similar_docs[0]
            
    print ('-'*80)
    buffer = "Title %2d         : %s" % (index, doc)
    print(buffer)
    print('Cosine similar   :', boblist[tsd[0]])
    print('Similarity score :', tsd[1])
    print ('-'*80) 
    print ("\n")

NameError: name 'normalize_corpus' is not defined